In [1]:
import numpy as np
import pandas as pd

In [2]:
tejdata = pd.read_csv('/smartbeta/smartbeta.csv')  
tejdata['yyyy'] = np.round(tejdata['日期']/100)
tejdata['coid'] = pd.DataFrame(tejdata['公司'].str.split(" ").tolist(),columns=['A','B'])['A']
tejdata['Momentum'] = tejdata['Momentum'].str.replace('%','').astype('float32')
tejdata['持有一年報酬率'] = tejdata['持有一年報酬率'].str.replace('%','').astype('float32')
tejdata['Momentum'] = tejdata['Momentum'] /100
from sklearn.preprocessing import quantile_transform
yyyy_list = tejdata['yyyy'].unique().tolist()

tejdata['總分'] = tejdata['總分'].fillna(0)
tejdata['總分'] = tejdata['總分'] +10
tejdata['Momentum'] = tejdata['Momentum'].fillna(0)

#報酬率資料處理
return_col_name = ['hold_Q1','hold_Q2','hold_Q3','hold_Q4','持有一年報酬率']
for i in range(0,len(return_col_name)):
    tejdata[return_col_name[i]] = tejdata[return_col_name[i]].fillna(0)
    tejdata[return_col_name[i]]  = tejdata[return_col_name[i]] /100
    tejdata.loc[tejdata[return_col_name[i]]>1,return_col_name[i]] = 1.0


In [3]:
train_col_name = ['淨值市價比','益本比(常續性利益)','現金股利率','現金流量模組指數','m_index','ROE(B)－常續利益']
roi_col_name = ['持有一年報酬率','總分']

n_max_steps = len(yyyy_list)
n_inputs = len(train_col_name)
sample_size = 500
x_batch = np.zeros((n_max_steps,sample_size,1,n_inputs))
y_batch = np.zeros((n_max_steps,sample_size,1))
z_batch = np.zeros((n_max_steps,sample_size))
from sklearn.preprocessing import StandardScaler    
scaler = StandardScaler()
for i in range(0,n_max_steps):
    this_year = tejdata[tejdata['yyyy']==yyyy_list[i]].reset_index()
    #計算動能分位數
    this_year['m_n'] = pd.DataFrame(quantile_transform(tejdata.loc[tejdata['yyyy']==yyyy_list[i],'Momentum'].values.reshape(1,-1),n_quantiles=3, random_state=0,axis =1).T.tolist())
    this_year['m_index'] = 1
    this_year.loc[this_year['m_n']<0.6,'m_index'] = 0
    this_year.loc[this_year['m_n']<0.3,'m_index'] = -1
    #計算總分排序
    this_year['total_score'] = 0
    for j in range(0,len(train_col_name)):
        this_year['total_score'] = this_year['total_score'] + this_year[train_col_name[j]]
#    this_year['score_n'] = 500*this_year['score_n']
    scaler.fit(this_year.reindex(columns=roi_col_name))
    this_year['score_r'] = pd.DataFrame(scaler.transform(this_year.reindex(columns=roi_col_name)),columns=roi_col_name)['總分']

    test_datas = this_year.reindex(columns=train_col_name).values
    x_batch[i,:,0] = test_datas
    roi = this_year.reindex(columns=['total_score']).values
    y_batch[i,:,0] = roi[:,0]
    roi = this_year.reindex(columns=['hold_Q1']).values
    z_batch[i] = roi[:,0]

In [4]:
import tensorflow as tf   

tf.reset_default_graph()   
sample_length = 30
punishment = 5

isTrain = False
batch_size = 1
learning_rate = 0.0005
optimizer = tf.train.AdamOptimizer(learning_rate) 
train_w_stddev = 0.3
keep_prob =1.0
# display_step = 10 #

n_step = sample_length # h
n_hidden = n_inputs
n_classes = 1 #跟group一樣多
num_layers = 3

# placeholder
x = tf.placeholder(tf.float32, [None, n_inputs])
y = tf.placeholder(tf.float32, [None, n_classes])
actual_y = tf.placeholder(tf.float32, [None, n_classes])

#w_in  = tf.Variable(tf.truncated_normal([n_inputs, n_hidden], mean=0.0, stddev=train_w_stddev, dtype=tf.float32))
w_out = tf.Variable(tf.truncated_normal([n_hidden, n_classes], mean=0.0, stddev=train_w_stddev, dtype=tf.float32))
#b_in  = tf.Variable(tf.zeros(n_hidden, dtype = tf.float32))
b_out = tf.Variable(tf.zeros(n_classes, dtype = tf.float32))

initializer = tf.contrib.layers.variance_scaling_initializer() 

#hidden  = tf.contrib.layers.fully_connected(tf.matmul(x     ,w_in )+b_in   ,n_hidden , activation_fn=tf.nn.relu,weights_initializer=initializer) 
logits  = tf.contrib.layers.fully_connected(tf.matmul(x,w_out)+b_out  ,n_classes, activation_fn=tf.nn.relu,weights_initializer=initializer) 

cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=actual_y, logits=logits)
cost = tf.reduce_mean(tf.square(logits -0.0))

###增強式學習用
grads_and_vars = optimizer.compute_gradients(cross_entropy) 
gradients = [variable for grad, variable in grads_and_vars] 
gradient_placeholders = [] 
grads_and_vars_feed = [] 
#計算平均梯度，避免梯度爆炸
for grads, variable in grads_and_vars:
#    X = tf.placeholder(tf.float32, shape=[None, n_inputs]) 
    gradient_placeholder = tf.placeholder(tf.float32, shape=variable.get_shape())
    gradient_placeholders.append(gradient_placeholder)
    grads_and_vars_feed.append((gradient_placeholder, variable)) 
training_op = optimizer.apply_gradients(grads_and_vars_feed) 

init = tf.global_variables_initializer() 
saver = tf.train.Saver() 

C:\Users\zyx\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [5]:
punishment = 5
#learning_rate = 0.001
n_iterations = 1000000     # number of training iterations 
     # max steps per episode 
n_games_per_update = n_max_steps # train the policy every 10 episodes 
save_iterations = 10    # save the model every 10 training iterations 
discount_rate = 1
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.6)
samples = []
ckpt_file = '/smartbeta/'
confident_level_0 = 0.97
confident_level_1 = 0.83
correct_rate=0.5000
break_rate=0.01909
cost_val = 0
choose_stock_num = 50
#config=tf.ConfigProto(gpu_options=gpu_options,log_device_placement=True)
with tf.Session() as sess:
    ckpt = tf.train.get_checkpoint_state(ckpt_file)
    if ckpt and ckpt.model_checkpoint_path:
        print('restoring...')
        saver.restore(sess, ckpt.model_checkpoint_path)
    else:
        print('not found')
        init.run()   
    if isTrain:
        print("start training")
    else:
        print("evaluating...")
        n_iterations = 1
        keep_prob=1.0       
    for iteration in range(n_iterations):
        count_1 = 0
        count_0 = 0                
        all_actions = []
        all_rewards = []    
        all_gradients = []  
        all_reward_test = []
        for game in range(3,n_games_per_update):  
            
            current_predict = 0
            current_rewards = 0
#            current_gradients = [] 
            current_reward_test = []
#                x_sample_index = np.random.randint(low=1,high=(allsample_length+1), size=1)[0]
#            for step in range(0,1):

            logits_val,cost_val,gradients_val = sess.run([logits,cost,gradients],feed_dict={actual_y:y_batch[game].reshape(-1,1),x: x_batch[game].reshape(-1,6)}) # one obs
#                print(str(logits_val)+" "+str(y_batch[game][step]))
            current_gradients = gradients_val            
            current_predict = logits_val[:,0]
            current_rewards = [1]*len(current_predict)
            temp_current_predict = np.sort(current_predict.copy())
            temp_current_predict = temp_current_predict[::-1]
            buy_index_logits = temp_current_predict[choose_stock_num-1]
            reward_count = 0
            for i in range(0,len(current_predict)):
                if current_predict[i]<buy_index_logits:
                    current_rewards[i]=0
                    current_reward_test.append(0)
                else:
#                    current_rewards[i]=1
                    a = (1+z_batch[game-1][i])*(1+z_batch[game-2][i])*(1+z_batch[game-3][i]) -1
                    current_rewards.append(a)
#                    current_rewards[i]=1
                    current_reward_test.append(z_batch[game][i])           
                    reward_count+=1
            mean_current_rewards = np.sum(current_rewards)/reward_count
            mean_current_rewards_test = np.sum(current_reward_test)/reward_count
            all_gradients.append(current_gradients)
            all_rewards.append(mean_current_rewards)
            all_reward_test.append(mean_current_rewards_test)
#            all_rewards = discount_and_normalize_rewards(all_rewards,discount_rate)
        feed_dict = {}
        for var_index, grad_placeholder in enumerate(gradient_placeholders):
            # 加工梯度
            this_mean = []
            for game_index, rewards in enumerate(all_rewards):
#                for step, reward in enumerate(rewards):            
                r_g = rewards * all_gradients[game_index][var_index]
                this_mean.append(r_g)
            mean_gradients = np.mean(this_mean,axis=0)
#            mean_gradients = np.mean([reward * all_gradients[game_index][step][var_index]for game_index, rewards in enumerate(all_rewards)for step, reward in enumerate(rewards)],axis=0)
            feed_dict[grad_placeholder] = mean_gradients        
        avg_retrun = np.mean(all_reward_test)
        if iteration%100==0:
            print(np.mean(all_reward_test))
        if avg_retrun>0.10:
            saver.save(sess, ckpt_file+"model_"+str(round(avg_retrun,6))+"_beta.ckpt")  
#            learning_rate = 0.0001
#        else:
#            learning_rate = 0.001
        if isTrain:
            apply = sess.run(training_op, feed_dict=feed_dict)          
        else:            
            w1,b1 = sess.run([w_out,b_out], feed_dict=feed_dict)        


not found
evaluating...
0.0027432468804841625
